<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2025/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

In [26]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

In [27]:
p=Props(['Benzene','Toluene'])
R=8.314 #J/(mol K)

In [28]:
z=np.array([0.4,0.6])

In [29]:
def bubbleP(x, T):
    Pi = x*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

In [30]:
def dewP(y, T):
    Pisat = p.Pvap(T)
    P=1/np.sum(y/Pisat)
    return P, y*P/Pisat

In [31]:
xs= np.linspace(0,1,21)
T=350.
Ps = []
y1s = []
for x in np.c_[xs , 1-xs ]:
    P, y = bubbleP(x, T)
    Ps.append(P)
    y1s.append(y[0])
Ps = np.r_[Ps]
y1s=np.r_[y1s]

In [32]:
fig = make_subplots()
fig.add_scatter(x=xs,y=Ps, name='bubble', mode='lines')
fig.add_scatter(x=y1s,y=Ps, name='dew', mode='lines')
fig.update_layout(width=800,height=600, title=f'Benzene-Tolune VLE at T={T} K', xaxis_title='x, y', yaxis_title='Pressure (Pa)')

In [33]:
p.HvapNB

array([30761.43, 33179.83])

In [34]:
p.Tbn

array([353.24, 383.78])

In [35]:
def T_estimate(P, x):
    return np.sum(x*1/(1/p.Tbn-R*np.log(P/101325)/p.HvapNB))

In [36]:
T_estimate(1e5, z)

np.float64(371.0955479288866)

In [37]:
def bubbleT(x, P):
    T = sp.optimize.root_scalar(lambda T: bubbleP(x, T)[0] - P, x0=T_estimate(P,x), method='Newton').root

    return T, x*p.Pvap(T)/P

In [38]:
def dewT(y, P):
    T = sp.optimize.root_scalar(lambda T: dewP(y, T)[0] - P, x0=T_estimate(P,x), method='Newton').root

    return T, y*P/p.Pvap(T)


In [39]:
dewT(z, 101325)

(Array(374.69027477, dtype=float64),
 Array([0.21613614, 0.78386386], dtype=float64))

In [40]:
xs= np.linspace(0,1,21)
Ts=[]
P=101325
y1s = []
for x in np.c_[xs , 1-xs ]:
    T, y = bubbleT(x, P)
    Ts.append(T)
    y1s.append(y[0])
Ts = np.r_[Ts]
y1s=np.r_[y1s]

In [41]:
fig = make_subplots(rows=1, cols=2)
fig.add_scatter(x=xs,y=Ts, name='bubble', mode='lines', row=1,col=1)
fig.add_scatter(x=y1s,y=Ts, name='dew', mode='lines', row=1,col=1)
fig.add_scatter(x=xs,y=y1s, mode='lines', row=1,col=2, name='y1')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', row=1,col=2, name='', line_color='grey')
fig.update_layout(width=800,height=400, title=f'Benzene-Tolune VLE at P={P} Pa', xaxis_title='x, y', yaxis_title='Temperature (K)')

In [42]:
Ts = np.linspace(300, 400, 11)

In [43]:
Pvaps = p.Pvap(Ts)
Pvaps

Array([[ 13759.39560528,   4180.76479093],
       [ 21316.15242598,   6809.24339458],
       [ 31964.10940927,  10701.84735345],
       [ 46550.5839087 ,  16288.96194041],
       [ 66036.08337891,  24086.35801651],
       [ 91487.72374629,  34697.50215991],
       [124070.9068048 ,  48813.71306907],
       [165040.21980094,  67212.39764337],
       [215730.48312433,  90753.7255339 ],
       [277548.76932456, 120376.18036806],
       [351968.0749818 , 157091.4610553 ]], dtype=float64)

In [44]:
Pvaps[:,0]/Pvaps[:,1]

Array([3.29111928, 3.13047297, 2.98678428, 2.85779929, 2.74163837,
       2.63672363, 2.54172238, 2.45550264, 2.37709782, 2.30567849,
       2.24052964], dtype=float64)

In [45]:
alpha = 2.5
x1=np.linspace(0,1,21)
y1 = alpha*x1/(1-x1+alpha*x1)

In [46]:
fig = make_subplots(rows=1, cols=1)
fig.add_scatter(x=x1,y=y1, mode='lines')
fig.add_scatter(x=[0,1],y=[0,1], mode='lines', line_color='grey')
fig.update_layout(width=500,height=500,showlegend=False)

In [69]:
P = 101325. #Pa
T = (dewT(z, P)[0] + bubbleT(z, P)[0])/2

def rachford_rice(z, P, T):
    K = p.Pvap(T)/P
    def rr0(phi):
        return np.sum(z/(1+phi*(K-1))) - 1
    phi = sp.optimize.root_scalar(rr0, x0 = 0.5, method = 'newton').root
    x = z/(1+phi*(K-1))
    y = K*x
    return (phi, x, y)

In [75]:
rachford_rice(z, P, 1.05*(dewT(z, P)[0]))

(Array(2.23177657e-17, dtype=float64),
 Array([0.4, 0.6], dtype=float64),
 Array([1.19039111, 0.78222799], dtype=float64))